In [1]:
train_batch_size = 64
val_batch_size = 32
num_workers = 4

num_epochs = 40

size_image = (128, 128)

learning_rate = 0.001

log_dir = './logs'
train_dir = '/home/paulbahush/bsuir_ml_part2/data/dogs-vs-cats/train'
test_dir = '/home/paulbahush/bsuir_ml_part2/data/dogs-vs-cats/test1'

In [2]:
import pandas as pd
import os
import torch
from torch.utils.tensorboard import SummaryWriter

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running")
else:
    device = torch.device("cpu")
    print("Running")

Running on the GPU


### Задание 1.
#### Загрузите данные. Разделите исходный набор данных на обучающую, валидационную и контрольную выборки.


In [4]:
files_labels = {os.path.join(train_dir, file_name): 0 if 'cat' in file_name else 1 
              for file_name in os.listdir(train_dir)}
data = pd.DataFrame.from_dict({'image_file': list(files_labels.keys()), 'label': list(files_labels.values())})

In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2)
test_data, val_data = train_test_split(test_data, test_size=0.25)

### Задание 2.
#### Реализуйте глубокую нейронную сеть с как минимум тремя сверточными слоями. Какое качество классификации получено?

In [6]:
from torch.utils.data import DataLoader
from torchvision import transforms


from torch.utils.data import Dataset
import numpy as np
from PIL import Image


class DogCatDataset(Dataset):
    def __init__(self, data, transforms=None):
        self.data = data
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.data.iloc[idx, 1]
        image_file = self.data.iloc[idx, 0]

        image  = Image.open(image_file)

        if self.transforms:
            image = self.transforms(np.uint8(image))
        else:
            image = image.astype(np.float32)

        return image, label


tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size_image),
    transforms.ToTensor()
])
train_dataset = DogCatDataset(train_data, transforms=tfms)
train_loader = DataLoader(train_dataset, num_workers=num_workers, batch_size=train_batch_size, shuffle=True)

val_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size_image),
    transforms.ToTensor()
])
val_dataset = DogCatDataset(val_data, transforms=val_tfms)
val_loader = DataLoader(val_dataset, num_workers=num_workers, batch_size=val_batch_size, shuffle=False)

test_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size_image),
    transforms.ToTensor()
])
test_dataset = DogCatDataset(test_data, transforms=test_tfms)
test_loader = DataLoader(test_dataset, num_workers=num_workers, batch_size=val_batch_size, shuffle=False)

In [7]:
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn as nn


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

        self.maxpool3 = nn.MaxPool2d(kernel_size=2)

        self.fc_preout = nn.Linear(4096, 100)
        self.fc = nn.Linear(100, 2)

    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)

        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)

        out = self.maxpool2(out)

        out = self.cnn3(out)
        out = self.relu3(out)

        out = self.maxpool3(out)

        out = out.view(out.size(0), -1)

        out = self.fc_preout(out)
        out = self.fc(out)

        return out

model = CNNModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 30)

In [8]:
def train(model, train_loader, test_loader, num_epochs,
    optimizer, criterion, logs_writer, scheduler, device):
    for epoch in tqdm(range(num_epochs)):

        for i, (images, labels) in enumerate(train_loader):

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            logs_writer.add_scalar('Itearation Loss/train', loss, epoch*len(train_loader) + i)

        accuracy = calculate_accuracy(model, test_loader, device)
        logs_writer.add_scalar('Accuracy/train', accuracy, epoch)
        logs_writer.add_scalar('Learning rate', np.array(scheduler.get_lr()), epoch)

train(model, train_loader, test_loader, num_epochs, 
    optimizer, criterion, logs_writer, exp_lr_scheduler, device)

100%|██████████| 40/40 [12:43<00:00, 19.09s/it]


### Задание 3.
#### Примените дополнение данных (data augmentation). Как это повлияло на качество классификатора?

In [9]:
from torch.utils.data import DataLoader
from torchvision import transforms


tfms = transforms.Compose([
        transforms.ToPILImage(mode=None),
        transforms.Resize(size_image),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3),
        transforms.ToTensor(),
    ])

train_dataset = DogCatDataset(train_data, transforms=tfms)
train_loader = DataLoader(train_dataset, num_workers=num_workers, batch_size=train_batch_size, shuffle=True)

In [10]:
import torch.nn as nn


model = CNNModel()
criterion = nn.CrossEntropyLoss()

model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 30)

In [11]:

train(model, train_loader, test_loader, num_epochs, 
    optimizer, criterion, logs_writer, exp_lr_scheduler, device)

100%|██████████| 40/40 [14:06<00:00, 21.17s/it]


### Задание 4.
#### Поэкспериментируйте с готовыми нейронными сетями (например, AlexNet, VGG16, Inception и т.п.), применив передаточное обучение. Как это повлияло на качество классификатора? Можно ли было обойтись без него? Какой максимальный результат удалось получить на контрольной выборке?


In [12]:
from torch.utils.data import DataLoader
from torchvision import transforms


tfms = transforms.Compose([
        transforms.ToPILImage(mode=None),
        transforms.Resize(size_image),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3),
        transforms.ToTensor(),
    ])

train_dataset = DogCatDataset(train_data, transforms=tfms)
train_loader = DataLoader(train_dataset, num_workers=num_workers, batch_size=train_batch_size, shuffle=True)

In [13]:
from torchvision import models
from torch.optim import lr_scheduler

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 30)


In [14]:
train(model, train_loader, val_loader, num_epochs,
    optimizer, criterion, logs_writer, exp_lr_scheduler, device)

100%|██████████| 40/40 [14:27<00:00, 21.69s/it]


In [15]:
from torchvision import models
from torch.optim import lr_scheduler

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 30)


In [16]:

train(model, train_loader, val_loader, num_epochs, 
    optimizer, criterion, logs_writer, exp_lr_scheduler, device)

100%|██████████| 40/40 [38:19<00:00, 57.48s/it]
